In [1]:
import transformers
from transformers import pipeline

pipe = pipeline(task='text-classification', model='ProsusAI/finbert')

Device set to use cpu


In [5]:
total_score = 0
num_articles = 0

In [6]:
import pandas as pd

file_path = fr'C:\Users\ivane\Desktop\Dissertation\data\maltabusinessweeklyArticles.csv'
df = pd.read_csv(fr'{file_path}')

df.head()

,headline,url,description,date
0,Gozo’s economic growth must continue to result...,https://maltabusinessweekly.com/gozos-economic...,Prime Minister Robert Abela said that Gozo is ...,2025-01-11T08:39:39+00:00
1,€1.6 billion to be invested in pensions and so...,https://maltabusinessweekly.com/e1-6-billion-t...,€1.6 billion is set to be spent on pensions an...,2025-01-10T16:45:00+00:00
2,Italy approves permit for the development of M...,https://maltabusinessweekly.com/italy-approves...,Italy's Environment Ministry has given its las...,2025-01-10T12:02:31+00:00
3,"Inflation, environment main concerns for Malte...",https://maltabusinessweekly.com/inflation-envi...,Inflation and the environment are the major co...,2024-12-19T14:51:34+00:00
4,How the stock market defied expectations again...,https://maltabusinessweekly.com/how-the-stock-...,What a wonderful year 2024 has been for invest...,2024-12-26T19:30:56+00:00


In [7]:
sentiment_label = []

for index, row in df.iterrows():
    print(f'Headline: {row["headline"]}')
    print(f'Date: {row["date"]}')
    print(f'Description: {row["description"]}')
    print(f'URL: {row["url"]}')
    
    sentiment = pipe(row['description'])[0]
    print(f'Sentiment: {sentiment}')
    
    print('-' * 40)
    if sentiment['label'] == 'positive':
        total_score += sentiment['score']
        num_articles += 1
    elif sentiment['label'] == 'negative':
        total_score -= sentiment['score']
        num_articles += 1
        
    sentiment_label.append(sentiment['label'])
    df.at[index, 'sentiment'] = sentiment['label']
    df.at[index, 'sentiment_score'] = sentiment['score']
    
df.to_csv(fr'{file_path}', index=False)

final_score = total_score / num_articles if num_articles > 0 else 0


Headline: Gozo’s economic growth must continue to result in common good and identity preservation, PM says
Date: 2025-01-11T08:39:39+00:00
Description: Prime Minister Robert Abela said that Gozo is an example of how economic growth must continue resulting in the common good, and the preservation of Maltese identity, speaking at the 25th anniversary of the Gozo Chamber of Commerce. During a celebration marking the 25th anniversary of the...
URL: https://maltabusinessweekly.com/gozos-economic-growth-must-continue-to-result-in-common-good-and-identity-preservation-pm-says/27972/
Sentiment: {'label': 'positive', 'score': 0.7338708639144897}
----------------------------------------
Headline: €1.6 billion to be invested in pensions and social benefits throughout 2025
Date: 2025-01-10T16:45:00+00:00
Description: €1.6 billion is set to be spent on pensions and social benefits throughout the calendar year of 2025, the government said. The Ministry for Social Policy said this mark will set "an a

In [8]:
# After your existing code, add:

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])
# Extract just the date part (removing time)
df['date'] = df['date'].dt.date

# Group by date and get majority sentiment
daily_sentiment = df.groupby('date')['sentiment'].agg(lambda x: x.value_counts().index[0]).reset_index()
# Get sentiment counts per day
daily_sentiment_counts = df.groupby(['date', 'sentiment']).size().unstack(fill_value=0)

# Print results
for date in daily_sentiment['date']:
    print(f'\nDate: {date}')
    print('Sentiment counts:')
    print(daily_sentiment_counts.loc[date])
    print(f"Majority sentiment: {daily_sentiment[daily_sentiment['date'] == date]['sentiment'].values[0]}")


Date: 2021-09-23
Sentiment counts:
sentiment
negative    0
neutral     0
positive    2
Name: 2021-09-23, dtype: int64
Majority sentiment: positive

Date: 2021-09-24
Sentiment counts:
sentiment
negative    0
neutral     1
positive    1
Name: 2021-09-24, dtype: int64
Majority sentiment: neutral

Date: 2021-09-27
Sentiment counts:
sentiment
negative    0
neutral     0
positive    1
Name: 2021-09-27, dtype: int64
Majority sentiment: positive

Date: 2021-09-28
Sentiment counts:
sentiment
negative    0
neutral     1
positive    1
Name: 2021-09-28, dtype: int64
Majority sentiment: neutral

Date: 2021-09-30
Sentiment counts:
sentiment
negative    0
neutral     1
positive    3
Name: 2021-09-30, dtype: int64
Majority sentiment: positive

Date: 2021-10-01
Sentiment counts:
sentiment
negative    0
neutral     0
positive    2
Name: 2021-10-01, dtype: int64
Majority sentiment: positive

Date: 2021-10-03
Sentiment counts:
sentiment
negative    0
neutral     1
positive    0
Name: 2021-10-03, dtype: i

In [ ]:
# 1. First, let's prepare our sentiment data
# Convert sentiment to numeric values for easier analysis
sentiment_mapping = {'positive': 1, 'negative': -1, 'neutral': 0}
df['sentiment_score_mapped'] = df['sentiment'].map(sentiment_mapping)

# Group by date to get daily sentiment metrics
daily_sentiment = df.groupby('date').agg({
    'sentiment_score_mapped': ['mean', 'count'],  # Average sentiment and number of articles
    'sentiment_score': ['mean', 'std']  # Mean and std dev of sentiment scores
}).reset_index()

# Flatten column names
daily_sentiment.columns = ['date', 'avg_sentiment', 'article_count', 'sentiment_score_mean', 'sentiment_score_std']

# 2. Load and prepare the stock price data
stock_df = pd.read_csv('path_to_your_stock_data.csv')  # Replace with your file path
stock_df['date'] = pd.to_datetime(stock_df['date'])

# 3. Merge sentiment and price data
merged_df = pd.merge(stock_df, daily_sentiment, on='date', how='left')

# 4. Handle missing values (days without news)
merged_df = merged_df.fillna({
    'avg_sentiment': 0,
    'article_count': 0,
    'sentiment_score_mean': 0,
    'sentiment_score_std': 0
})

# 5. Create lagged features (previous day's values)
merged_df['prev_day_sentiment'] = merged_df['avg_sentiment'].shift(1)
merged_df['prev_day_price'] = merged_df['price'].shift(1)
merged_df['prev_day_pct_change'] = merged_df['percentage_change'].shift(1)

# 6. Create target variable (next day's price movement)
merged_df['next_day_pct_change'] = merged_df['percentage_change'].shift(-1)

# 7. Prepare features for modeling
features = ['prev_day_sentiment', 'sentiment_score_mean', 'article_count', 
           'prev_day_price', 'prev_day_pct_change']
target = 'next_day_pct_change'

# Remove rows with NaN values
merged_df = merged_df.dropna()

In [ ]:


# 8. Split data into training and testing sets
from sklearn.model_selection import train_test_split

X = merged_df[features]
y = merged_df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# 9. Train a model (example using Random Forest)
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 10. Make predictions and evaluate
predictions = model.predict(X_test)

print("Model Performance Metrics:")
print(f"R2 Score: {r2_score(y_test, predictions):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, predictions)):.4f}")

# 11. Feature importance analysis
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

# 12. Optional: Visualize results
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(y_test.index, y_test.values, label='Actual')
plt.plot(y_test.index, predictions, label='Predicted')
plt.title('Actual vs Predicted Price Changes')
plt.legend()
plt.show()

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.bar(feature_importance['feature'], feature_importance['importance'])
plt.xticks(rotation=45)
plt.title('Feature Importance')
plt.tight_layout()
plt.show()